In [ ]:
1+1

In [ ]:
import matplotlib.pyplot as plt
import random

In [ ]:
%matplotlib inline

In [ ]:
plt.plot([random.random() for _ in range(20)])

--------------------------------


# Import des Données

## Données Trello

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import seaborn as sns
%matplotlib inline

dict_drey = json.load(open('boardDreyfus.json'))

In [ ]:
dict_drey.keys()

In [ ]:
df_cards= json_normalize(dict_drey['cards'])

In [ ]:
df_cards[['name', 'idShort' , 'desc', 'url']].head()

In [ ]:
df_cards.sort_values('dateLastActivity', inplace=True)

In [ ]:
df_cards['one']= 0
df_cards.loc[df_cards.subscribed,'one'] = df_cards[df_cards.subscribed].name.map(lambda x: x[1] if x[1] in ('1', '2', '3', '4', '5', '6') else '1').map(int).cumsum()
df_cards.dateLastActivity = pd.to_datetime((df_cards.dateLastActivity), format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
df_cards.set_index('dateLastActivity').one.iloc[40:].plot()

In [ ]:
defaults_ids = set([l['id'] for l in dict_drey['labels'] if l['color']=='red'])
list_done = [l['id'] for l in dict_drey['lists'] if l['name'].lower().find('done')>-1]
df_cards= json_normalize(dict_drey['cards'])
df_tickets = df_cards[df_cards['idList'].isin(list_done)]
bac_rouge = df_tickets[df_tickets['idLabels'].map(lambda x: len(set(x)&defaults_ids)> 0)]
bac_rouge.dateLastActivity = pd.to_datetime(bac_rouge.dateLastActivity)
df_tickets['is_default'] = df_tickets['idLabels'].map(lambda x: len(set(x)&defaults_ids)> 0)
df_tickets.dateLastActivity = pd.to_datetime(df_tickets.dateLastActivity , format='%Y-%m-%d %H:%M:%S.%f')
df_tickets= df_tickets[df_tickets.name.map(lambda x:x.find('(')>-1)]
df_tickets[['is_default', 'name']].groupby('is_default').count()
df_tickets['estim'] = df_tickets.name.map(lambda x: x[1])
df_tickets = df_tickets[df_tickets['estim'].isin([u'3', u'2', u'1',u'4'])]
df_tickets['estim'] =df_tickets['estim'].map(int)
df_tickets['post_eval'] = df_tickets.name.map(lambda x: int(x[-2]) if x[-1]==']' else -1)
df_tickets = df_tickets.sort('dateLastActivity')
df_tickets['is_default'] = df_tickets['is_default'] | (df_tickets['post_eval']>df_tickets['estim'])
df_tickets['is_default'] = df_tickets['is_default'].map(int)
df_tickets['one']=1
df_tickets['ticketnb'] = df_tickets['one'].cumsum()
df_tickets['defaultnb'] = df_tickets['is_default'].cumsum()
df_tickets[df_tickets.set_index('dateLastActivity').index>"2016.12"].set_index('dateLastActivity')[['ticketnb', 'defaultnb']].plot(figsize = (15,15))

## Données GIT

In [ ]:
import commands

In [ ]:
print commands.getstatusoutput(' cd ~/app2/dreyfus && git log | head -n20')[1]

In [ ]:
print commands.getstatusoutput(' cd ~/app2/dreyfus && git diff-tree --no-commit-id --name-only -r e885904aa6d18327d9f82fcc393feec07d9f5fd1')[1]

In [ ]:
with open('commit_hash_filechanged.json') as f:
    commits_file = json.load(f)

with open('commit_hash_ticket.json') as f:
    commits_tickets = json.load(f)

df_co_ti = pd.DataFrame([commits_tickets.keys(), commits_tickets.values()]).T
dfco_fc = pd.read_json('commit_hash_filechanged.json', orient = 'index')

df_co_ti[1] = df_co_ti[1].map(lambda x:x[1:-1])

dfco_fc.drop(0,1, inplace = True)

df_co_ti.columns = ['hash', 'ticket']

dfco_fc.columns = ['files']

df_files = df_co_ti.set_index('hash').join(dfco_fc)

df_files.head()

## Join data

In [ ]:
with open('commit_hash_filechanged.json') as f:
    commits_file = json.load(f)
with open('commit_hash_ticket.json') as f:
    commits_tickets = json.load(f)
df_co_ti = pd.DataFrame([commits_tickets.keys(), commits_tickets.values()]).T
dfco_fc = pd.read_json('commit_hash_filechanged.json', orient = 'index')
df_co_ti[1] = df_co_ti[1].map(lambda x:x[1:-1])
dfco_fc.drop(0,1, inplace = True)
df_co_ti.head()
df_co_ti.columns = ['hash', 'ticket']
dfco_fc.columns = ['files']
dfco_fc.head()
df_files = df_co_ti.set_index('hash').join(dfco_fc)
df_files.shape
df_files.drop_duplicates(['ticket']).shape
df_files.head()
df_files.dropna(subset = ['ticket']).shape
df_files.groupby('ticket').count().sort_values('files', ascending = 0)
dffoo = df_files.groupby('ticket').agg({'files': lambda x:list(x)})
df_tickets.idShort =df_tickets.idShort.map(str)
df_tickets = df_tickets.join(dffoo, on='idShort')

In [ ]:
df_tickets[['name', 'idShort' , 'desc', 'url', 'files']].head()